## **Ecobici data**

Trabajando con datos abiertos del gobierno de CABA referidos a recorridos/viajes realizados en las biciclietas del sistema de Bike Sharing de CABA, "EcoBici". Los datos comprenden el periodo 2010-2019.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

import pandas as pd
import json

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("ecobici") \
        .getOrCreate()

Los archivos no mantienen un esquema homogéneo entre sí. Tienen diferentes números y nombres de columna.

Previamente, se ha trabajado para descartar algunas columnas y obtener un archivo JSON con los nombres de las columnas para uno de los archivos.

In [3]:
with open("files_schema.json", "r") as json_file:
    columns_per_files = json.load(json_file)
columns_per_files

{'recorridos-realizados-2010.csv': ['fecha_origen_recorrido',
  'id_estacion_origen',
  'long_estacion_origen',
  'lat_estacion_origen',
  'duracion_recorrido',
  'fecha_destino_recorrido',
  'id_estacion_destino',
  'long_estacion_destino',
  'lat_estacion_destino'],
 'recorridos-realizados-2011.csv': ['fecha_origen_recorrido',
  'id_estacion_origen',
  'long_estacion_origen',
  'lat_estacion_origen',
  'duracion_recorrido',
  'fecha_destino_recorrido',
  'id_estacion_destino',
  'long_estacion_destino',
  'lat_estacion_destino'],
 'recorridos-realizados-2012.csv': ['fecha_origen_recorrido',
  'id_estacion_origen',
  'lat_estacion_origen',
  'long_estacion_origen',
  'duracion_recorrido',
  'fecha_destino_recorrido',
  'id_estacion_destino',
  'lat_estacion_destino',
  'long_estacion_destino'],
 'recorridos-realizados-2013.csv': ['fecha_origen_recorrido',
  'id_estacion_origen',
  'long_estacion_origen',
  'lat_estacion_origen',
  'duracion_recorrido',
  'fecha_destino_recorrido',
  '

### **Null values**
Primero contaremos los valores nulos para cada una de las columnas de cada archivo

In [4]:
for file, columns in columns_per_files.items():
    print(file)
    rides = spark.read.csv(file, header=True)
    rides = rides.select(*columns)
    for column in columns:
        print(f"Column: {column}\t Nulls: {rides.filter(F.isnull(column)).count()}")

recorridos-realizados-2010.csv
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 0
Column: long_estacion_origen	 Nulls: 0
Column: lat_estacion_origen	 Nulls: 0
Column: duracion_recorrido	 Nulls: 3
Column: fecha_destino_recorrido	 Nulls: 0
Column: id_estacion_destino	 Nulls: 0
Column: long_estacion_destino	 Nulls: 0
Column: lat_estacion_destino	 Nulls: 0
recorridos-realizados-2011.csv
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 85
Column: long_estacion_origen	 Nulls: 85
Column: lat_estacion_origen	 Nulls: 85
Column: duracion_recorrido	 Nulls: 148
Column: fecha_destino_recorrido	 Nulls: 0
Column: id_estacion_destino	 Nulls: 178
Column: long_estacion_destino	 Nulls: 178
Column: lat_estacion_destino	 Nulls: 178
recorridos-realizados-2012.csv
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 2487
Column: lat_estacion_origen	 Nulls: 2487
Column: long_estacion_origen	 Nulls: 2487
Column: duracion_recorrido	 Nu

El análisis de estos valores nulos se puede separar de acuerdo a grupos de columnas.

Con respecto a los campos ***id_estacion***, ***lat*** y ***long*** (tanto para la estación origen como destino): Mantienen el mismo número de valores nulos entre sí, podemos borrar aquellas filas con valores nulos para el campo id_estacion (ya sea origen o destino) ya que sin ese id no es posible obtener otros datos de la estacion.

Sobre los campos ***fechas*** y ***duración*** del recorrido: Todos los archivos tienen una fecha de origen del recorrido. Si hay filas con valores nulos para los campos duracion y fecha de destino del recorrido, se descartan. Si solo una de esas fuese nula, es posible calcular mediante las funciones que correspondan.

### **Reordering columns**

Hay un archivo que presenta un nombre diferente a los demás para los campos sobre *fecha*, se lo corregirá para que coincida con el nombre de los otros archivos.
Luego, con respecto a los campos de *latitud* y *longitud*, no mantienen el mismo orden entre los archivos, algunos presentan *(lat, long)* y otros *(long, lat)*, también se corregirá para que se presentan en este orden *(lat, long)*

In [4]:
for file, columns in columns_per_files.items():
    print(file, \
          list(filter(lambda x: x.startswith("origen") or \
                      x.startswith("destino"), columns)))

recorridos-realizados-2010.csv []
recorridos-realizados-2011.csv []
recorridos-realizados-2012.csv []
recorridos-realizados-2013.csv []
recorridos-realizados-2014.csv ['origen_recorrido_fecha', 'destino_recorrido_fecha']
recorridos-realizados-2015.csv []
recorridos-realizados-2016.csv []
recorridos-realizados-2017.csv []
recorridos-realizados-2018.csv []
recorridos-realizados-2019.csv []


In [5]:
ordered_columns = columns_per_files["recorridos-realizados-2019.csv"] # Columnas ordenadas que todos los archivos deben seguir

In [6]:
for fname in columns_per_files.keys():
    rides = spark.read.csv(fname, header=True)
    if fname.endswith("2014.csv"):
        rides = rides.withColumnRenamed("origen_recorrido_fecha", \
                                        "fecha_origen_recorrido") \
                    .withColumnRenamed("destino_recorrido_fecha", \
                                       "fecha_destino_recorrido")
    rides = rides.select(*ordered_columns)
    tablename = f"rides{fname[-8:-4]}" # 'rides' string + year
    rides.write.saveAsTable(tablename)

### **Deleting null values**

In [7]:
table_names = [f"rides201{i}" for i in range(10)]

In [8]:
for tname in table_names:
    print(tname)
    rides = spark.table(tname)
    
    # Deleting rows about origin station
    cols = rides.columns[1:4]
    print(f"Columns to inspect: {cols}")
    rides = rides.dropna(how="all", subset=cols)
    
    # Deleting rows about target station
    cols = rides.columns[6:]
    print(f"Columns to inspect: {cols}")
    rides = rides.dropna(how="all", subset=cols)
    
    # Deleting rows about date and duration ride
    cols = rides.columns[4:6]
    print(f"Columns to inspect: {cols}")
    rides = rides.dropna(how="all", subset=cols)
    
    for column in columns:
        print(f"Column: {column}\t Nulls: {rides.filter(F.isnull(column)).count()}")
    
    # Dataframe to table
    rides.write.saveAsTable("temp")
    spark.table("temp") \
            .write.insertInto(tname, overwrite=True)
    spark.sql("DROP TABLE IF EXISTS temp")

rides2010
Columns to inspect: ['id_estacion_origen', 'lat_estacion_origen', 'long_estacion_origen']
Columns to inspect: ['id_estacion_destino', 'lat_estacion_destino', 'long_estacion_destino']
Columns to inspect: ['duracion_recorrido', 'fecha_destino_recorrido']
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 0
Column: lat_estacion_origen	 Nulls: 0
Column: long_estacion_origen	 Nulls: 0
Column: duracion_recorrido	 Nulls: 3
Column: fecha_destino_recorrido	 Nulls: 0
Column: id_estacion_destino	 Nulls: 0
Column: lat_estacion_destino	 Nulls: 0
Column: long_estacion_destino	 Nulls: 0
rides2011
Columns to inspect: ['id_estacion_origen', 'lat_estacion_origen', 'long_estacion_origen']
Columns to inspect: ['id_estacion_destino', 'lat_estacion_destino', 'long_estacion_destino']
Columns to inspect: ['duracion_recorrido', 'fecha_destino_recorrido']
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 0
Column: lat_estacion_origen	 Nulls: 0
Column:

### **Filling nulls values**

In [15]:
def change_column_dtype(df, column, new_dtype):
    df = df.withColumn(column, \
                             df[column] \
                              .cast(new_dtype) \
                              .alias(column))
    return df

Se cuenta con un dataset con datos como id, nombre, latitud, longitud, direccion, etc. de cada estacion de ecobici.

In [10]:
stations = spark.read.csv("estaciones-bicicletas-publicas.csv", header=True)
stations = stations.repartition(1)
# stations.rdd.getNumPartitions()
print(*stations.columns)

lat long nombre_estacion id_estacion capacidad dirección_completa direccion_nombre direccion_altura direccion_interseccion barrio


In [11]:
stations = change_column_dtype(stations, "id_estacion", "int")
stations.select("id_estacion").dtypes

[('id_estacion', 'int')]

Primero trabajaremos con los campos referidos a **latitud** y **longitud**, donde solo los archivos de los años 2015 y 2019 presentaban valores nulos en dichos campos.

In [12]:
for year in ["2015", "2019"]:
    print(f"rides{year}")
    rides = spark.table(f"rides{year}")
    rides.filter(F.isnull("lat_estacion_origen") & \
                F.isnull("long_estacion_origen")) \
        .select("id_estacion_origen").distinct().show()
    
    rides.filter(F.isnull("lat_estacion_destino") & \
                F.isnull("long_estacion_destino")) \
        .select("id_estacion_destino").distinct().show()

rides2015
+------------------+
|id_estacion_origen|
+------------------+
|                15|
|                39|
+------------------+

+-------------------+
|id_estacion_destino|
+-------------------+
|                 15|
|                 39|
+-------------------+

rides2019
+------------------+
|id_estacion_origen|
+------------------+
|             159_0|
|              44_0|
+------------------+

+-------------------+
|id_estacion_destino|
+-------------------+
|              159_0|
|               44_0|
+-------------------+



Respecto a los particulares ids "159_0" y "44_0", los verificaremos con el dataset de estaciones

In [13]:
stations.filter((stations.id_estacion == 15) | (stations.id_estacion == 39) | \
               (stations.id_estacion == 159) | (stations.id_estacion == 44) | \
               (stations.id_estacion == "44_0") | (stations.id_estacion == "159_0")) \
        .select("id_estacion").show()

+-----------+
|id_estacion|
+-----------+
|         15|
|         39|
|         44|
+-----------+



La estacion de id "159_0" o 159 no existe, se puede descartar las filas con ese valor.
Luego, nos aseguramos que el valor de id "44_0", corresponde a 44.

In [14]:
spark.read.csv("recorridos-realizados-2019.csv", header=True) \
.filter("id_estacion_origen = '44_0'") \
.select("nombre_estacion_origen").show(1)

stations.filter("id_estacion == 44").select("nombre_estacion").show()

+----------------------+
|nombre_estacion_origen|
+----------------------+
|             Ecoparque|
+----------------------+
only showing top 1 row

+---------------+
|nombre_estacion|
+---------------+
|044 - Ecoparque|
+---------------+



Efectivamente, coinciden. Ahora continuamos con los tratamientos correspondientes

In [15]:
rides = spark.table("rides2019")
rides = rides.replace("44_0", "44", subset=["id_estacion_origen", "id_estacion_destino"])
rides = change_column_dtype(rides, "id_estacion_origen", "int")
rides = change_column_dtype(rides, "id_estacion_destino", "int")
rides = rides.na.drop(how="any", subset=["id_estacion_origen", "id_estacion_destino"])
rides.write.saveAsTable("temp")
spark.table("temp").write.insertInto("rides2019", overwrite=True)
_ = spark.sql("DROP TABLE IF EXISTS temp")

Finalmente, ya podemos completar los valores nulos para los campos de latidad y longitud

In [16]:
def filling_geographical_fields(rides_df, stations_df, coord_type, station_type):
    """
    coord_type: lat/long
    station_type: origen/destino
    """
    stations_df = stations_df.select(*["id_estacion", "lat", "long"])
    stations_df = stations_df.withColumnRenamed("id_estacion", \
                                                f"id_estacion_{station_type}")
    
    joined_df = rides_df.join(stations_df, on=f"id_estacion_{station_type}")
    joined_df = joined_df.withColumn(f"{coord_type}_estacion_{station_type}", \
                                F.coalesce(joined_df[f"{coord_type}_estacion_{station_type}"], \
                                          joined_df[coord_type]))
    joined_df = joined_df.drop("lat", "long")
    return joined_df

In [17]:
for year in ["2015", "2019"]:
    rides = spark.table(f"rides{year}")
    ordered_cols = rides.columns
    for station_type in ["origen", "destino"]:
        for coord_type in ["lat", "long"]:
            rides = filling_geographical_fields(rides, stations, coord_type, station_type)
            
    print(f"rides{year}")
    for column in rides.columns:
        if 'lat' in column or 'long' in column:
            print(f"Column: {column}\t Nulls: {rides.filter(F.isnull(column)).count()}")
    
    rides = rides.select(ordered_columns)
    rides.write.saveAsTable("temp", mode="overwrite")
    spark.table("temp").write.insertInto(f"rides{year}", overwrite=True)
    spark.sql("DROP TABLE temp")
    

rides2015
Column: lat_estacion_origen	 Nulls: 0
Column: long_estacion_origen	 Nulls: 0
Column: lat_estacion_destino	 Nulls: 0
Column: long_estacion_destino	 Nulls: 0
rides2019
Column: lat_estacion_origen	 Nulls: 0
Column: long_estacion_origen	 Nulls: 0
Column: lat_estacion_destino	 Nulls: 0
Column: long_estacion_destino	 Nulls: 0


#### **Fechas y duracion**
Hechemos un vistazo al formato de los datos de fecha y duración

In [10]:
for tname in table_names:
    print(tname)
    rides = spark.read.parquet(f"spark-warehouse2/{tname}/*")
    rides.write.saveAsTable(tname)

rides2010
rides2011
rides2012
rides2013
rides2014
rides2015
rides2016
rides2017
rides2018
rides2019


In [18]:
for table in table_names:
    rides = spark.table(table)
    date_columns = list(filter(lambda column: 'fecha' in column, rides.columns))
    dates = rides.select(date_columns+['duracion_recorrido'])
    print(table)
    dates.show(5)

rides2010
+----------------------+-----------------------+------------------+
|fecha_origen_recorrido|fecha_destino_recorrido|duracion_recorrido|
+----------------------+-----------------------+------------------+
|      30/12/2010 19:39|       30/12/2010 19:46|               7.0|
|      30/12/2010 19:34|       30/12/2010 19:47|              13.0|
|      30/12/2010 19:10|       30/12/2010 19:17|               7.0|
|      30/12/2010 19:03|       30/12/2010 19:29|              26.0|
|      30/12/2010 19:02|       30/12/2010 19:13|              11.0|
+----------------------+-----------------------+------------------+
only showing top 5 rows

rides2011
+----------------------+-----------------------+------------------+
|fecha_origen_recorrido|fecha_destino_recorrido|duracion_recorrido|
+----------------------+-----------------------+------------------+
|   2011-09-29 19:40:00|       29/09/2011 19:49|              10.0|
|   2011-09-29 19:37:00|       29/09/2011 19:57|              20.0|
|  

Formatos los valores de campos sobre fecha

In [12]:
def format_date(column):
    formats = ["yyyy-MM-dd HH:mm", "dd/MM/yyyy HH:mm"]
    return F.coalesce(F.to_timestamp(column, formats[0]), \
                     F.to_timestamp(column, formats[1]))

In [11]:
def diff_in_minutes(start_date, end_date):
    """
    Return the difference in minutes between end_date and start_date
    """
    diff = (F.col(end_date).cast("long") - F.col(start_date).cast("long")) / 60
    return diff

In [18]:
def duration_in_minutes(df, duration_col):
    """
    turn ride duration in minutes
    """
    df = df.withColumn("duration_days", \
                      F.split(duration_col, "days").getItem(0))
    df = df.withColumn("hour", \
                      F.split(duration_col, "days").getItem(1))
    df = df.withColumn("hour", \
                      F.split("hour", ":"))
    df = df.withColumn(duration_col, \
                      F.col("duration_days")*1440 + \
                      F.col("hour").getItem(0)*60 + \
                       F.col("hour").getItem(1))
    df = df.drop("duration_days", "hour")
    df = df.withColumn(duration_col, \
                      F.col(duration_col).cast("int"))
    return df

In [92]:
for tname in table_names:
    print(tname)
    rides = spark.table(tname)
    
    for id_estacion in ["id_estacion_origen", "id_estacion_destino"]:
        rides = change_column_dtype(rides, id_estacion, "int")
    
    for date_col in ["fecha_origen_recorrido", "fecha_destino_recorrido"]:
        rides = rides.withColumn(date_col, \
                                format_date(date_col))
        
    # Calcular la duracion del recorrido donde hay valores nulos
    if tname in ["rides2010", "rides2011", "rides2012"]:
        rides = rides.withColumn("duracion_recorrido", \
                                F.coalesce("duracion_recorrido", \
                                          diff_in_minutes("fecha_origen_recorrido", \
                                                         "fecha_destino_recorrido")))
        rides = change_column_dtype(rides, "duracion_recorrido", "int")
    
    else:
        rides = duration_in_minutes(rides, "duracion_recorrido")
    
    # Calcular la fecha de destino donde hay valores nulos
    if tname in ["rides2016", "rides2017"]:
        rides = rides.withColumn("fecha_destino_recorrido", \
                         F.coalesce("fecha_destino_recorrido", \
                                   F.from_unixtime( \
                                                   F.expr("to_unix_timestamp(fecha_origen_recorrido, 'yyyy-MM-dd HH:mm:ss') \
                                                   + (duracion_recorrido * 60)")).cast("timestamp")))   
    
    rides.write.option("path", "/home/jovyan/ecobici_rides/tables/") \
        .saveAsTable("all_tables", mode="append")

rides2010
rides2011
rides2012
rides2013
rides2014
rides2015
rides2016
rides2017
rides2018
rides2019


Separamos en campos diferentes la fecha y la hora del recorrido